In [2]:
import json
import urllib
import sys
import os
import time
import re
import hashlib

_sha256 = '[a-z0-9]{64}' # sha256에 맞는 정규표현식 정의

path_dir = 'C:/OfficeSetup.exe'
file_list = os.listdir(path_dir)

sha256_pattern = re.compile(_sha256)

class vtAPI():
    def __init__(self):
        self.api = 'dbae55ce231bd44e635b5310d293fa247b2c98c8ee5664b8100a3fb65b035fb6'
        self.base = 'https://www.virustotal.com/vtapi/v2/'
        
    # Virustotal에 주어진 악성코드가 검사된 적이 있는지 확인
    def getReport(self,sha256):
        param = {'resource':sha256,'apikey':self.api,'allinfo': '1'}
        url = self.base + "file/report"
        data = urllib.parse.urlencode(param) # Python 3에는 urllib가 나뉘어져 있음 (parse, request)
        data = data.encode('utf-8') # urlopen()에 data를 넣기 위해, 인코딩이 되어 있어야 함.
        result = urllib.request.urlopen(url,data)

        jdata = json.loads(result.read())
        print(jdata)

        if jdata['response_code'] == 0:
            print(sha256 + " -- Not Found in VT")
            return "no"
        else:
            print("=== Results for SHA256: ", jdata['sha256'], "\tDetected by: ", jdata['positives'])
            return jdata['positives']

    # Virustotal에 악성코드 검사를 의뢰함
    def reqScan(self,filepath):
        print("- Requesting a new scan")
        param = {'file':filepath,'apikey':self.api}
        url = self.base + "file/scan"
        data = urllib.parse.urlencode(param) # Python 3에는 urllib가 나뉘어져 있음 (parse, request)
        data = data.encode('utf-8') # urlopen()에 data를 넣기 위해, 인코딩이 되어 있어야 함.
        result = urllib.request.urlopen(url,data)

        jdata = json.loads(result.read())

        return jdata
    
    # sha256해쉬값을 구함
    def getSha256(self, filepath, blocksize=8192):
        sha256 = hashlib.sha256()
        try:
            f = open(filepath, "rb")
        except IOError as e:
            print("file open error", e)
            return
        while True:
            buf = f.read(blocksize)
            if not buf:
                break
            sha256.update(buf)
        return sha256.hexdigest()


def main():
    vt = vtAPI()
    i = 0
    for file in file_list:
        before = path_dir + "/" + file
        name_check = re.search(sha256_pattern, file) # 악성코드 파일명이 sha256형식에 맞는지 확인

        if name_check == None:
            file = vt.getSha256(before)


        i += 1
        rns = vt.getReport(file) #rns는 악성코드가 백신에 탐지된 갯수를 의미함

        if(rns == "no"): # Virustotal에 악성코드 검사 이력이 없을 때
            file_path = os.getcwd() + "/" + file
            rns = vt.reqScan(file_path)
            file = rns['sha256']

            while True: # 이후 검사이력이 생성될때까지 실행
                time.sleep(20)
                rns = vt.getReport(file)
                if(rns != "no"):
                    break

        after = path_dir + "/" + str(rns) + "#" + file # 본 코드를 처리한 악성코드의 이름 변경

        print("Processed " + str(i) + " files - "+ after)
        os.rename(before, after)

        time.sleep(15)

if __name__ == '__main__':
    main()

{'scans': {'Bkav': {'detected': False, 'version': '1.3.0.9899', 'result': None, 'update': '20210105'}, 'Lionic': {'detected': False, 'version': '4.2', 'result': None, 'update': '20210105'}, 'Elastic': {'detected': True, 'version': '4.0.14', 'result': 'malicious (high confidence)', 'update': '20201214'}, 'DrWeb': {'detected': True, 'version': '7.0.49.9080', 'result': 'Trojan.Solimba.55', 'update': '20210105'}, 'MicroWorld-eScan': {'detected': True, 'version': '14.0.409.0', 'result': 'Trojan.GenericKD.35924501', 'update': '20210105'}, 'FireEye': {'detected': True, 'version': '32.36.1.0', 'result': 'Generic.mg.0dcf5cfe19363ce9', 'update': '20210105'}, 'CAT-QuickHeal': {'detected': True, 'version': '14.00', 'result': 'PUA.Appsinstal.Gen', 'update': '20210105'}, 'ALYac': {'detected': True, 'version': '1.1.3.1', 'result': 'Trojan.GenericKD.35924501', 'update': '20210105'}, 'Cylance': {'detected': True, 'version': '2.3.1.101', 'result': 'Unsafe', 'update': '20210105'}, 'Zillya': {'detected': 

{'scans': {'Bkav': {'detected': True, 'version': '1.3.0.9899', 'result': 'W32.BarackF.Trojan', 'update': '20211123'}, 'Lionic': {'detected': False, 'version': '4.2', 'result': None, 'update': '20211123'}, 'Elastic': {'detected': True, 'version': '4.0.31', 'result': 'malicious (high confidence)', 'update': '20211118'}, 'MicroWorld-eScan': {'detected': True, 'version': '14.0.409.0', 'result': 'Trojan.Ransomware.GenericKD.32177572', 'update': '20211123'}, 'FireEye': {'detected': True, 'version': '32.44.1.0', 'result': 'Generic.mg.2337089f52251079', 'update': '20211123'}, 'CAT-QuickHeal': {'detected': False, 'version': '14.00', 'result': None, 'update': '20211123'}, 'ALYac': {'detected': True, 'version': '1.1.3.1', 'result': 'Trojan.Ransomware.GenericKD.32177572', 'update': '20211124'}, 'Cylance': {'detected': True, 'version': '2.3.1.101', 'result': 'Unsafe', 'update': '20211124'}, 'VIPRE': {'detected': True, 'version': '97172', 'result': 'Trojan.Win32.Generic!BT', 'update': '20211123'}, '

{'scans': {'Bkav': {'detected': False, 'version': '1.3.0.9899', 'result': None, 'update': '20210116'}, 'Lionic': {'detected': False, 'version': '4.2', 'result': None, 'update': '20210118'}, 'Elastic': {'detected': False, 'version': '4.0.15', 'result': None, 'update': '20210107'}, 'MicroWorld-eScan': {'detected': False, 'version': '14.0.409.0', 'result': None, 'update': '20210118'}, 'FireEye': {'detected': False, 'version': '32.36.1.0', 'result': None, 'update': '20210118'}, 'CAT-QuickHeal': {'detected': False, 'version': '14.00', 'result': None, 'update': '20210118'}, 'McAfee': {'detected': False, 'version': '6.0.6.653', 'result': None, 'update': '20210118'}, 'Cylance': {'detected': False, 'version': '2.3.1.101', 'result': None, 'update': '20210118'}, 'Zillya': {'detected': False, 'version': '2.0.0.4272', 'result': None, 'update': '20210116'}, 'Paloalto': {'detected': False, 'version': '1.0', 'result': None, 'update': '20210118'}, 'K7AntiVirus': {'detected': True, 'version': '11.160.36

{'scans': {'Bkav': {'detected': False, 'version': '1.3.0.9899', 'result': None, 'update': '20210105'}, 'Lionic': {'detected': True, 'version': '4.2', 'result': 'Trojan.Win32.Blocker.j!c', 'update': '20210105'}, 'Elastic': {'detected': True, 'version': '4.0.14', 'result': 'malicious (high confidence)', 'update': '20201214'}, 'MicroWorld-eScan': {'detected': True, 'version': '14.0.409.0', 'result': 'Gen:Variant.Razy.628274', 'update': '20210105'}, 'FireEye': {'detected': True, 'version': '32.36.1.0', 'result': 'Generic.mg.0bcc615d4cc4ba9c', 'update': '20210105'}, 'CAT-QuickHeal': {'detected': False, 'version': '14.00', 'result': None, 'update': '20210105'}, 'ALYac': {'detected': True, 'version': '1.1.3.1', 'result': 'Gen:Variant.Razy.628274', 'update': '20210105'}, 'Cylance': {'detected': True, 'version': '2.3.1.101', 'result': 'Unsafe', 'update': '20210105'}, 'VIPRE': {'detected': True, 'version': '89444', 'result': 'Trojan.Win32.Generic!BT', 'update': '20210105'}, 'Paloalto': {'detecte

{'scans': {'Bkav': {'detected': False, 'version': '1.3.0.10239', 'result': None, 'update': '20190820'}, 'Lionic': {'detected': True, 'version': '4.2', 'result': 'Trojan.Win32.Autoit.4!c', 'update': '20190821'}, 'TotalDefense': {'detected': True, 'version': '37.1.62.1', 'result': 'Win32/Inject.NCcWETD', 'update': '20190821'}, 'MicroWorld-eScan': {'detected': True, 'version': '14.0.297.0', 'result': 'Gen:Trojan.Heur.AutoIT.13', 'update': '20190821'}, 'FireEye': {'detected': True, 'version': '29.7.0.0', 'result': 'Generic.mg.e2d80fd725b6959d', 'update': '20190821'}, 'CAT-QuickHeal': {'detected': False, 'version': '14.00', 'result': None, 'update': '20190820'}, 'McAfee': {'detected': True, 'version': '6.0.6.653', 'result': 'Artemis!E2D80FD725B6', 'update': '20190821'}, 'Cylance': {'detected': True, 'version': '2.3.1.101', 'result': 'Unsafe', 'update': '20190821'}, 'Paloalto': {'detected': True, 'version': '1.0', 'result': 'generic.ml', 'update': '20190821'}, 'K7AntiVirus': {'detected': Tru

...

{'scans': {'Bkav': {'detected': False, 'version': '1.3.0.9899', 'result': None, 'update': '20210105'}, 'Lionic': {'detected': False, 'version': '4.2', 'result': None, 'update': '20210107'}, 'Elastic': {'detected': True, 'version': '4.0.14', 'result': 'malicious (high confidence)', 'update': '20201214'}, 'MicroWorld-eScan': {'detected': False, 'version': '14.0.409.0', 'result': None, 'update': '20210105'}, 'FireEye': {'detected': True, 'version': '32.36.1.0', 'result': 'Generic.mg.0b1056caf259f471', 'update': '20210105'}, 'CAT-QuickHeal': {'detected': True, 'version': '14.00', 'result': 'PUA.Wedownload1.Gen', 'update': '20210105'}, 'Qihoo-360': {'detected': True, 'version': '1.0.0.1120', 'result': 'Win32/Virus.Downloader.a96', 'update': '20210105'}, 'ALYac': {'detected': False, 'version': '1.1.3.1', 'result': None, 'update': '20210105'}, 'Cylance': {'detected': True, 'version': '2.3.1.101', 'result': 'Unsafe', 'update': '20210105'}, 'VIPRE': {'detected': True, 'version': '89444', 'resul

{'scans': {'Bkav': {'detected': True, 'version': '1.3.0.9899', 'result': 'W32.FamVT.InstallQTTc.Worm', 'update': '20210318'}, 'Lionic': {'detected': True, 'version': '4.2', 'result': 'Trojan.Multi.Generic.muwi', 'update': '20210318'}, 'Elastic': {'detected': True, 'version': '4.0.17', 'result': 'malicious (high confidence)', 'update': '20210217'}, 'MicroWorld-eScan': {'detected': True, 'version': '14.0.409.0', 'result': 'Gen:Application.Bundler.InstallIQ.1', 'update': '20210318'}, 'FireEye': {'detected': True, 'version': '32.44.1.0', 'result': 'Generic.mg.e6e63552ba14e5f7', 'update': '20210318'}, 'CAT-QuickHeal': {'detected': True, 'version': '14.00', 'result': 'PUA.Secureinst1.Gen', 'update': '20210318'}, 'McAfee': {'detected': True, 'version': '6.0.6.653', 'result': 'GenericRXAK-JZ!E6E63552BA14', 'update': '20210318'}, 'Cylance': {'detected': True, 'version': '2.3.1.101', 'result': 'Unsafe', 'update': '20210319'}, 'Zillya': {'detected': True, 'version': '2.0.0.4318', 'result': 'Adwar

{'scans': {'Bkav': {'detected': False, 'version': '1.3.0.10239', 'result': None, 'update': '20190925'}, 'Lionic': {'detected': False, 'version': '4.2', 'result': None, 'update': '20190926'}, 'MicroWorld-eScan': {'detected': True, 'version': '14.0.297.0', 'result': 'Gen:Variant.Ulise.53186', 'update': '20190926'}, 'CMC': {'detected': False, 'version': '1.1.0.977', 'result': None, 'update': '20190321'}, 'CAT-QuickHeal': {'detected': True, 'version': '14.00', 'result': 'PUA.Optimumins.Gen', 'update': '20190925'}, 'McAfee': {'detected': True, 'version': '6.0.6.653', 'result': 'Adware-FOO', 'update': '20190926'}, 'Cylance': {'detected': True, 'version': '2.3.1.101', 'result': 'Unsafe', 'update': '20190926'}, 'Zillya': {'detected': True, 'version': '2.0.0.3909', 'result': 'Adware.AgentCRT.Win32.988', 'update': '20190925'}, 'K7AntiVirus': {'detected': True, 'version': '11.68.32097', 'result': 'Adware ( 005524301 )', 'update': '20190926'}, 'Alibaba': {'detected': True, 'version': '0.3.0.5', 'r

file open error [Errno 21] Is a directory: '/home/user/Desktop/dataset/malware/.fr-IOx6je'
{'response_code': 0, 'resource': 'None', 'verbose_msg': 'Invalid resource, check what you are submitting'}


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'